In [2]:
'''
用于对一次实验中的多路iFEMG信号进行特征提取

没有计算主动与拮抗肌肉之间的差值'''

import pandas as pd
import os

from gui_model_utils import form_feature_df
from iFEMG_feature import read_label

In [3]:
'''
获取文件夹路径中的所有文件名
'''
folder_path = r"D:\code\data\iFEMG_data_set\Three_channel_dataset\chw20230315"
file_name_list = []
for file in os.listdir(folder_path):
    file_name_list.append(file.split('.')[0])
    pass
# print(file_name_list)
print(set(file_name_list))

{'20230315_chw', 'bicps-05kg', 'bicps-0kg', 'bicps-2kg', 'bicps-1kg'}


In [4]:
"""
init dataframe
REVISE BEFORE RUNNING:  
    1. ch_name_list
    2. ch_str
"""
feature_name_list = ['subject_name', 'height(cm)', 'weight(kg)', 'gender', 'age', 'sensor_channel', 'label(kg)', 
                    'FMG_increase', 'mav', 'rms', 'wave_length', 'zero_crossing', 'slope_sign_change', 'mean_freq', 'mean_power_freq']

ch1_str = "agonist_ch1"
ch2_str = "antagonist_ch1"
ch3_str = "antagonist_ch2"

ch1_name_list = []
ch2_name_list = []
ch3_name_list = []
for i in feature_name_list:
    ch1_name_list.append(ch1_str)
    ch2_name_list.append(ch2_str)
    ch3_name_list.append(ch3_str)

all_feature_df = pd.DataFrame(columns=pd.MultiIndex.from_product([[ch1_str, ch2_str, ch3_str], feature_name_list]))
#all_feature_df = pd.DataFrame(columns=[ch1_name_list + ch2_name_list + ch3_name_list, feature_name_list + feature_name_list + feature_name_list])
print(all_feature_df)
print("Init dataframe shape: ", all_feature_df.shape)

Empty DataFrame
Columns: [(agonist_ch1, subject_name), (agonist_ch1, height(cm)), (agonist_ch1, weight(kg)), (agonist_ch1, gender), (agonist_ch1, age), (agonist_ch1, sensor_channel), (agonist_ch1, label(kg)), (agonist_ch1, FMG_increase), (agonist_ch1, mav), (agonist_ch1, rms), (agonist_ch1, wave_length), (agonist_ch1, zero_crossing), (agonist_ch1, slope_sign_change), (agonist_ch1, mean_freq), (agonist_ch1, mean_power_freq), (antagonist_ch1, subject_name), (antagonist_ch1, height(cm)), (antagonist_ch1, weight(kg)), (antagonist_ch1, gender), (antagonist_ch1, age), (antagonist_ch1, sensor_channel), (antagonist_ch1, label(kg)), (antagonist_ch1, FMG_increase), (antagonist_ch1, mav), (antagonist_ch1, rms), (antagonist_ch1, wave_length), (antagonist_ch1, zero_crossing), (antagonist_ch1, slope_sign_change), (antagonist_ch1, mean_freq), (antagonist_ch1, mean_power_freq), (antagonist_ch2, subject_name), (antagonist_ch2, height(cm)), (antagonist_ch2, weight(kg)), (antagonist_ch2, gender), (antago

In [5]:
"""
read data
NEED TO CHANGE BEFORE RUNNING:
    1. file_name_list1 文件名
    2. label_list 数据的重量标签
    3. subject_arg_input 受试者的其他特征参数
    4. signal_channel_list 有用信号所在的通道号，ch_name_list 主动肌或拮抗肌，channel_name = "bicps_br"肌肉名称对应。三者需对应
    5. abandon_ms = 1000, 抛弃前一秒数据，尽量确保数据是等长收缩状态
"""
# folder_path = r"D:\code\data\iFEMG_data_set\Three_channel_dataset\zpk\bicps_br"
# 'bicps-1kg', 'tricps-0-5kg', 'tricps-0kg', 'tricps-1kg', 'bicps-0-5kg', 'bicps-2kg', 'bicps-0kg'
file_name_list1 = ['bicps-0kg', 'bicps-05kg', 'bicps-1kg', 'bicps-2kg'] # 文件名
label_list1 = [0, 0.5, 1, 2]   # 数据的重量标签
subject_arg_input = {"subject_height": 165,
                    "subject_weight": 55,
                    "subject_age": 22,
                    "subject_gender": 1,
                    "subject_name": "chw"}


# signal_channel, 信号对应的传感器通道
signal_channel_list = [1,3,5]

for i in range(len(file_name_list1)):
    #raw_data = pd.read_table(folder_path+'\\'+file_name_list1[i]+'.db', sep = ';', header = None)
    #label = read_label(folder_path+'\\'+file_name_list1[i]+'.txt')
    temp_df1 = form_feature_df(db_file_path=folder_path+'\\'+file_name_list1[i]+'.db',
                            time_file_path=folder_path+'\\'+file_name_list1[i]+'.txt',
                            signal_channel=signal_channel_list[0],
                            channel_name="bicps_br",
                            abandon_ms=1000,
                            strength_level=label_list1[i],
                            **subject_arg_input)
    temp_df1.columns=[ch1_name_list, feature_name_list]
    # medial长头
    temp_df2 = form_feature_df(db_file_path=folder_path+'\\'+file_name_list1[i]+'.db',
                            time_file_path=folder_path+'\\'+file_name_list1[i]+'.txt',
                            signal_channel=signal_channel_list[2],
                            channel_name="tricps_br_medial",
                            abandon_ms=1000,
                            strength_level=label_list1[i],
                            **subject_arg_input)
    temp_df2.columns=[ch3_name_list, feature_name_list]
    # lateral外侧头
    temp_df3 = form_feature_df(db_file_path=folder_path+'\\'+file_name_list1[i]+'.db',
                            time_file_path=folder_path+'\\'+file_name_list1[i]+'.txt',
                            signal_channel=signal_channel_list[1],
                            channel_name="tricps_br_lateral",
                            abandon_ms=1000,
                            strength_level=label_list1[i],
                            **subject_arg_input)
    temp_df3.columns=[ch2_name_list, feature_name_list]
    # 横向拼接三个通道特征dataframe
    temp_df = pd.concat([temp_df3, temp_df2, temp_df1], axis=1)
    print(temp_df.shape)
    # 数据加入all_feature_df
    all_feature_df = pd.concat([all_feature_df, temp_df], axis=0, ignore_index=True)
    print(all_feature_df.shape)
    pass


all_feature_df.head

(5, 45)
(5, 45)
(6, 45)
(11, 45)
(5, 45)
(16, 45)
(5, 45)
(21, 45)


<bound method NDFrame.head of     agonist_ch1                                                       \
   FMG_increase age gender height(cm) label(kg)       mav  mean_freq   
0      0.706537  22      1        165         0 -0.027074  40.864611   
1      0.873850  22      1        165         0 -0.556567  21.892985   
2      0.854075  22      1        165         0  2.607554  23.814285   
3      0.708636  22      1        165         0  1.675169  18.912624   
4      0.587577  22      1        165         0 -0.024988  19.062642   
5      0.678722  22      1        165       0.5  6.821509  19.974891   
6      0.512566  22      1        165       0.5  0.332138  21.705918   
7      0.533036  22      1        165       0.5 -0.156201  26.275030   
8      0.482861  22      1        165       0.5 -0.178406  26.425804   
9      0.460275  22      1        165       0.5 -0.021166  20.460788   
10     0.409448  22      1        165       0.5  0.360403  18.583743   
11     0.537434  22      1        

In [15]:
all_feature_df.to_csv(r"D:\code\data\iFEMG_data_set\Three_channel_dataset\chw20230315\chw_bicps_br.csv")

In [6]:
"""
model training
"""
import numpy as np
import time
from sklearn import preprocessing
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [7]:
data_df = pd.read_csv(r"D:\code\data\iFEMG_data_set\Three_channel_dataset\LiRuya\Lry_tricps_br_lay.csv", header=[0, 1], index_col=0)
print(data_df.shape)
data_df.head()

(21, 45)


agonist_ch1                                                        \
  FMG_increase age gender height(cm) label(kg)       mav   mean_freq   
0     0.937523  26      1        190       0.0 -0.021555  227.663721   
1     0.612404  26      1        190       0.0  0.021675  203.328556   
2     0.449468  26      1        190       0.0 -0.017833  211.940052   
3     0.503179  26      1        190       0.0  0.007088  208.515139   
4     0.525926  26      1        190       0.0  0.006271  200.694340   

                                            ... antagonist_ch2             \
  mean_power_freq       rms sensor_channel  ...            mav  mean_freq   
0      230.672803 -0.013744       bicps_br  ...      -0.686966  20.251347   
1      218.865884  0.020377       bicps_br  ...       0.539267  18.996560   
2      226.000617 -0.014693       bicps_br  ...       0.439427  26.999719   
3      223.503587  0.005871       bicps_br  ...      -0.494998  27.232816   
4      218.401315  0.010724       bicps_br  ...      -0.776704  25.770866   

                                                                              \
  mean_power_freq       rms    sensor_channel slope_sign_change subject_name   
0       31.135772 -0.421613  tricps_br_medial          0.765831        zpk01   
1       37.263596  0.324382  tricps_br_medial         -0.034871        zpk01   
2       32.168350  0.233613  tricps_br_medial          0.013359        zpk01   
3       31.726462 -0.182515  tricps_br_medial          0.115808        zpk01   
4       36.873994 -0.744217  tricps_br_medial          0.292858        zpk01   

                                        
  wave_length weight(kg) zero_crossing  
0   -0.581063         80      2.409131  
1    0.366878         80     -0.320178  
2    0.131993         80     -0.305509  
3   -0.591457         80     -0.157006  
4   -0.761045         80     -0.116219  

[5 rows x 45 columns]

In [8]:
x1_data = data_df["agonist_ch1"][['height(cm)', 'weight(kg)', 'gender', 'age', 'FMG_increase', 'mav', 'rms', 'wave_length', 'zero_crossing', 'slope_sign_change', 'mean_freq', 'mean_power_freq']].values
x2_data = data_df["agonist_ch2"][['height(cm)', 'weight(kg)', 'gender', 'age', 'FMG_increase', 'mav', 'rms', 'wave_length', 'zero_crossing', 'slope_sign_change', 'mean_freq', 'mean_power_freq']].values
x3_data = data_df["antagonist_ch2"][['height(cm)', 'weight(kg)', 'gender', 'age', 'FMG_increase', 'mav', 'rms', 'wave_length', 'zero_crossing', 'slope_sign_change', 'mean_freq', 'mean_power_freq']].values

y_data = data_df["agonist_ch1"]['label(kg)'].values
x_data = np.concatenate((x1_data, x2_data, x3_data), axis=1)
print("data shape: ", x_data.shape)

train_data_r, test_data_r, train_label, test_label = train_test_split(x_data, y_data, test_size = 0.2, random_state = 0)
# train_data = preprocessing.normalize(train_data_r, norm = 'max')
# test_data = preprocessing.normalize(test_data_r, norm = 'max')
scaler = preprocessing.StandardScaler().fit(train_data_r)
train_data = scaler.transform(train_data_r)
test_data = scaler.transform(test_data_r)

print("train data shape: ", train_data.shape)
print("train label length: ", train_label.shape)
print("test data shape: ", test_data.shape)
print("test label length: ", test_label.shape)

KeyError: 'agonist_ch2'

In [ ]:
"""
Grid searching best parameters of SVR model
"""
# 记录开始训练时间
start_time = time.time()
print("start training, time: ", time.strftime("%Y--%m--%d %H:%M:%S", time.localtime(start_time)))

# 自动选择合适的参数
svr = GridSearchCV(SVR(), param_grid={"kernel": ("rbf", "linear"), "C": np.logspace(-3, 3, 7), "gamma": np.logspace(-3, 3, 7)}, n_jobs=-1)
svr.fit(train_data, train_label)

print(svr.best_params_)

end_time = time.time()
print("finish, time: ", time.strftime("%Y--%m--%d %H:%M:%S", time.localtime(end_time)))

start training, time:  2022--11--21 21:22:39
{'C': 1.0, 'gamma': 0.1, 'kernel': 'rbf'}
finish, time:  2022--11--21 21:22:40


In [ ]:
score_test = svr.score(test_data, test_label)
score_train = svr.score(train_data, train_label)
print(str(svr) + "train score： " + str(score_train))
print(str(svr) + "test score： " + str(score_test))
test_pre = svr.predict(test_data)
train_pre = svr.predict(train_data)
print("train mean squared error: ", mean_squared_error(train_label, train_pre))
print("test mean squared error: ", mean_squared_error(test_label, test_pre))


GridSearchCV(estimator=SVR(), n_jobs=-1,
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'gamma': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'kernel': ('rbf', 'linear')})train score： 0.9492612112237954
GridSearchCV(estimator=SVR(), n_jobs=-1,
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'gamma': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'kernel': ('rbf', 'linear')})test score： 0.9475451019066321
train mean squared error:  0.008495207450243339
test mean squared error:  0.006362872253929249
